# QC-Py-13 - Alpha Models et Algorithm Framework

> **Architecture modulaire pour strategies quantitatives**
> Duree: 75 minutes | Niveau: Intermediaire-Avance | Python + QuantConnect

---

## Objectifs d'Apprentissage

A la fin de ce notebook, vous serez capable de :

1. Comprendre l'**Algorithm Framework** et ses 5 composants modulaires
2. Creer et utiliser des **Insights** pour communiquer les predictions
3. Utiliser les **Alpha Models built-in** (EMA Cross, MACD, RSI)
4. Developper des **Custom Alpha Models** avec gestion des donnees par symbole
5. Combiner plusieurs modeles avec **CompositeAlphaModel**
6. Construire une **strategie complete** avec le Framework

## Prerequisites

- Notebooks QC-Py-01 a 12 completes
- Comprehension des indicateurs techniques (RSI, MACD, EMA)
- Familiarite avec Universe Selection et Portfolio Management

## Structure du Notebook

1. Algorithm Framework Overview (15 min)
2. Comprendre les Insights (20 min)
3. Alpha Models Built-In (25 min)
4. Custom Alpha Models (30 min)
5. Composite Alpha (15 min)
6. Strategie Complete avec Framework (20 min)

---

## Partie 1 : Algorithm Framework Overview (15 min)

### Qu'est-ce que l'Algorithm Framework?

L'**Algorithm Framework** de QuantConnect est une architecture modulaire qui separe la logique de trading en 5 composants independants. Cette approche offre :

- **Reutilisabilite** : Chaque composant peut etre reutilise dans differentes strategies
- **Testabilite** : Tester chaque module isolement
- **Separation des responsabilites** : Chaque composant a un role bien defini
- **Collaboration** : Differentes personnes peuvent travailler sur differents modules

### Les 5 Composants du Framework

```
                    Algorithm Framework
                           |
     +---------------------+---------------------+
     |                     |                     |
     v                     v                     v
+----------+         +-----------+        +------------+
| Universe |  --->   |   Alpha   |  --->  | Portfolio  |
| Selection|         |   Model   |        |Construction|
+----------+         +-----------+        +------------+
                           |                     |
                           v                     v
                    +------------+        +------------+
                    |    Risk    |  <---  | Execution  |
                    | Management |        |   Model    |
                    +------------+        +------------+
```

| Composant | Role | Methode Set |
|-----------|------|-------------|
| **Universe Selection** | Definir quels actifs sont disponibles | `SetUniverseSelection()` |
| **Alpha Model** | Generer des predictions (Insights) | `SetAlpha()` |
| **Portfolio Construction** | Convertir Insights en poids cibles | `SetPortfolioConstruction()` |
| **Execution Model** | Executer les ordres | `SetExecution()` |
| **Risk Management** | Appliquer des contraintes de risque | `SetRiskManagement()` |

### Flux de donnees

```
Market Data --> Universe Selection --> Securities disponibles
                     |
                     v
            Alpha Model --> Insights (predictions)
                     |
                     v
        Portfolio Construction --> Target Weights
                     |
                     v
            Execution Model --> Orders
                     |
                     v
           Risk Management --> Adjusted Orders
```

In [ ]:
# Imports QuantConnect pour le notebook
from AlgorithmImports import *

# Imports standard
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

print("Imports reussis")
print("\nAlgorithm Framework Components:")
print("  1. Universe Selection - Quels actifs trader")
print("  2. Alpha Model - Predictions de marche")
print("  3. Portfolio Construction - Allocation du capital")
print("  4. Execution Model - Execution des ordres")
print("  5. Risk Management - Contraintes de risque")

### Structure d'un Algorithm Framework

Voici la structure de base d'un algorithme utilisant le Framework :

In [ ]:
# Structure de base d'un Algorithm Framework
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *

class BasicFrameworkAlgorithm(QCAlgorithm):
    """
    Exemple minimal d'utilisation de l'Algorithm Framework.
    Montre comment configurer les 5 composants.
    """
    
    def Initialize(self):
        # Configuration de base
        self.SetStartDate(2022, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        # === 1. UNIVERSE SELECTION ===
        # Definit quels actifs sont disponibles pour le trading
        # Ici: une liste manuelle de 5 actions tech
        symbols = [Symbol.Create(ticker, SecurityType.Equity, Market.USA) 
                   for ticker in ["AAPL", "MSFT", "GOOGL", "AMZN", "META"]]
        self.SetUniverseSelection(ManualUniverseSelectionModel(symbols))
        
        # === 2. ALPHA MODEL ===
        # Genere des predictions (Insights) sur la direction du marche
        # Ici: Alpha Model base sur croisement EMA 12/26
        self.SetAlpha(EmaCrossAlphaModel(
            fastPeriod=12,
            slowPeriod=26,
            resolution=Resolution.Daily
        ))
        
        # === 3. PORTFOLIO CONSTRUCTION ===
        # Convertit les Insights en allocations de portfolio
        # Ici: allocation egale entre tous les actifs avec Insight actif
        self.SetPortfolioConstruction(EqualWeightingPortfolioConstructionModel())
        
        # === 4. EXECUTION MODEL ===
        # Execute les ordres generes par Portfolio Construction
        # Ici: execution immediate au prix du marche
        self.SetExecution(ImmediateExecutionModel())
        
        # === 5. RISK MANAGEMENT ===
        # Applique des contraintes de risque (stop-loss, position limits)
        # Ici: modele null (pas de contraintes additionnelles)
        self.SetRiskManagement(NullRiskManagementModel())
        
        self.Log("Framework Algorithm initialized with all 5 components")

print("BasicFrameworkAlgorithm defini")
print("\nLes 5 methodes Set:")
print("  1. SetUniverseSelection(model)")
print("  2. SetAlpha(model)")
print("  3. SetPortfolioConstruction(model)")
print("  4. SetExecution(model)")
print("  5. SetRiskManagement(model)")

### Avantages du Framework vs Approche Traditionnelle

| Aspect | Approche Traditionnelle | Algorithm Framework |
|--------|------------------------|--------------------|
| **Structure** | Code monolithique dans `OnData()` | Composants separes |
| **Reutilisabilite** | Faible (copier-coller) | Elevee (plug & play) |
| **Tests** | Difficile a isoler | Chaque module testable |
| **Maintenance** | Complexe | Simple |
| **Collaboration** | Difficile | Chacun son module |
| **Complexite initiale** | Faible | Moyenne |

### Quand utiliser le Framework?

- **Strategies multi-actifs** avec selection dynamique
- **Systemes de production** necessitant maintenabilite
- **Combinaison de signaux** (plusieurs Alpha Models)
- **Backtests extensifs** avec variations de composants

Pour les strategies simples sur un seul actif, l'approche traditionnelle `OnData()` peut etre plus directe.

---

## Partie 2 : Comprendre les Insights (20 min)

### Qu'est-ce qu'un Insight?

Un **Insight** est une prediction sur un actif financier. C'est le format standard de communication entre l'Alpha Model et le Portfolio Construction Model.

Un Insight contient :

| Propriete | Type | Description |
|-----------|------|-------------|
| `Symbol` | Symbol | L'actif concerne |
| `Direction` | InsightDirection | Up, Down, ou Flat |
| `Period` | TimeSpan | Duree de validite de la prediction |
| `Magnitude` | float | Amplitude attendue (optionnel) |
| `Confidence` | float | Niveau de confiance 0-1 (optionnel) |
| `Weight` | float | Poids pour l'allocation (optionnel) |

### InsightDirection

```
InsightDirection.Up   = 1   : Prediction haussiere (acheter)
InsightDirection.Down = -1  : Prediction baissiere (vendre/short)
InsightDirection.Flat = 0   : Pas de direction claire (neutre)
```

### Types d'Insights

| Methode | Description |
|---------|-------------|
| `Insight.Price()` | Prediction sur le prix (le plus courant) |
| `Insight.Volatility()` | Prediction sur la volatilite |
| `Insight.Group()` | Groupe d'insights lies |

In [ ]:
# Demonstration de creation d'Insights
# Ces exemples montrent la syntaxe

from AlgorithmImports import *
from datetime import timedelta

# Exemple conceptuel (ne peut pas s'executer sans contexte QC)
print("=" * 60)
print("CREATION D'INSIGHTS - Exemples de syntaxe")
print("=" * 60)

print("\n1. Insight Price basique:")
print("""
insight = Insight.Price(
    symbol,                     # L'actif (Symbol object)
    timedelta(days=5),          # Duree: 5 jours
    InsightDirection.Up         # Direction: haussiere
)
""")

print("2. Insight avec Magnitude et Confidence:")
print("""
insight = Insight.Price(
    symbol,
    timedelta(days=5),
    InsightDirection.Up,
    0.02,                       # Magnitude: +2% attendu
    0.75                        # Confidence: 75%
)
""")

print("3. Insight avec Weight pour allocation:")
print("""
insight = Insight.Price(
    symbol,
    timedelta(days=5),
    InsightDirection.Up,
    0.02,
    0.75,
    None,                       # Source model (optionnel)
    0.25                        # Weight: 25% du portfolio
)
""")

print("4. Insight baissier (Short):")
print("""
insight = Insight.Price(
    symbol,
    timedelta(days=10),
    InsightDirection.Down,      # Baissier
    0.05,                       # -5% attendu
    0.60                        # 60% confiance
)
""")

print("5. Insight neutre (Flat):")
print("""
insight = Insight.Price(
    symbol,
    timedelta(days=1),
    InsightDirection.Flat       # Pas de direction
)
""")

### Lifecycle d'un Insight

```
1. CREATION (Alpha Model)
   |
   +-- Alpha Model detecte un signal
   +-- Cree un Insight avec direction, duree, confiance
   |
2. TRANSMISSION
   |
   +-- L'Insight est envoye au Portfolio Construction Model
   |
3. UTILISATION (Portfolio Construction)
   |
   +-- Convertit Insights en target weights
   +-- Utilise Direction, Magnitude, Confidence pour allocation
   |
4. EXPIRATION
   |
   +-- Apres la Period, l'Insight expire
   +-- Le Portfolio Construction peut liquider la position
```

### Comment le Portfolio Construction utilise les Insights

| Model | Utilisation des Insights |
|-------|-------------------------|
| **EqualWeighting** | Ignore Magnitude/Confidence, allocation egale |
| **InsightWeighting** | Poids bases sur Weight property |
| **ConfidenceWeighted** | Poids proportionnels a Confidence |
| **MeanVariance** | Optimisation Markowitz avec Magnitude comme expected return |

In [ ]:
# Demonstration pratique: Alpha Model simple qui cree des Insights
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *

class InsightDemoAlgorithm(QCAlgorithm):
    """
    Demonstration de creation d'Insights manuellement.
    Cree des insights bases sur le momentum.
    """
    
    def Initialize(self):
        self.SetStartDate(2023, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        # Ajouter des actifs
        self.symbols = [
            self.AddEquity("SPY", Resolution.Daily).Symbol,
            self.AddEquity("QQQ", Resolution.Daily).Symbol,
            self.AddEquity("IWM", Resolution.Daily).Symbol
        ]
        
        # Indicateurs pour chaque symbole
        self.momentum = {}
        for symbol in self.symbols:
            self.momentum[symbol] = self.MOMP(symbol, 20, Resolution.Daily)
        
        # Warmup
        self.SetWarmup(20)
    
    def OnData(self, data):
        if self.IsWarmingUp:
            return
        
        insights = []
        
        for symbol in self.symbols:
            if not self.momentum[symbol].IsReady:
                continue
            
            mom_value = self.momentum[symbol].Current.Value
            
            # Creer un Insight base sur le momentum
            if mom_value > 5:  # Momentum positif fort
                insight = Insight.Price(
                    symbol,
                    timedelta(days=5),
                    InsightDirection.Up,
                    mom_value / 100,  # Magnitude en pourcentage
                    min(mom_value / 20, 1.0)  # Confidence basee sur force
                )
                insights.append(insight)
                self.Log(f"UP Insight: {symbol.Value}, Mom={mom_value:.2f}, Conf={insight.Confidence:.2f}")
            
            elif mom_value < -5:  # Momentum negatif fort
                insight = Insight.Price(
                    symbol,
                    timedelta(days=5),
                    InsightDirection.Down,
                    abs(mom_value) / 100,
                    min(abs(mom_value) / 20, 1.0)
                )
                insights.append(insight)
                self.Log(f"DOWN Insight: {symbol.Value}, Mom={mom_value:.2f}, Conf={insight.Confidence:.2f}")
        
        # Emettre les insights (le framework les traite)
        if insights:
            self.EmitInsights(insights)

print("InsightDemoAlgorithm defini")
print("\nPoints cles:")
print("  - Insight.Price(symbol, period, direction, magnitude, confidence)")
print("  - self.EmitInsights(list_of_insights) pour les emettre")
print("  - Le Portfolio Construction Model recoit ces Insights")

---

## Partie 3 : Alpha Models Built-In (25 min)

QuantConnect fournit plusieurs Alpha Models prets a l'emploi. Ces modeles implementent des strategies classiques d'analyse technique.

### Alpha Models disponibles

| Model | Base | Signal |
|-------|------|--------|
| `EmaCrossAlphaModel` | Croisement EMA | Fast EMA croise Slow EMA |
| `MacdAlphaModel` | MACD | Croisement MACD/Signal line |
| `RsiAlphaModel` | RSI | Surachat/Survente |
| `ConstantAlphaModel` | - | Insight constant (benchmark) |
| `HistoricalReturnsAlphaModel` | Momentum | Rendement passe |

### 3.1 EmaCrossAlphaModel

L'**EmaCrossAlphaModel** genere des Insights bases sur le croisement de deux EMAs :

- **Signal haussier** : EMA rapide croise au-dessus de l'EMA lente
- **Signal baissier** : EMA rapide croise en-dessous de l'EMA lente

In [ ]:
# EmaCrossAlphaModel - Croisement de moyennes mobiles exponentielles
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *

class EmaCrossAlgorithm(QCAlgorithm):
    """
    Strategie utilisant EmaCrossAlphaModel.
    - Fast EMA: 12 periodes
    - Slow EMA: 26 periodes
    """
    
    def Initialize(self):
        self.SetStartDate(2022, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        # Universe: tech stocks
        symbols = [Symbol.Create(ticker, SecurityType.Equity, Market.USA) 
                   for ticker in ["AAPL", "MSFT", "GOOGL", "NVDA", "AMD"]]
        self.SetUniverseSelection(ManualUniverseSelectionModel(symbols))
        
        # === EMA CROSS ALPHA MODEL ===
        # Parametres:
        #   fastPeriod: Periode de l'EMA rapide (defaut: 12)
        #   slowPeriod: Periode de l'EMA lente (defaut: 26)
        #   resolution: Resolution des donnees (defaut: Daily)
        self.SetAlpha(EmaCrossAlphaModel(
            fastPeriod=12,
            slowPeriod=26,
            resolution=Resolution.Daily
        ))
        
        # Portfolio et Execution
        self.SetPortfolioConstruction(EqualWeightingPortfolioConstructionModel())
        self.SetExecution(ImmediateExecutionModel())
        self.SetRiskManagement(NullRiskManagementModel())
        
        self.Log("EMA Cross Algorithm initialized")
        self.Log("  Fast EMA: 12, Slow EMA: 26")
        self.Log("  UP Insight: Fast crosses above Slow")
        self.Log("  DOWN Insight: Fast crosses below Slow")

print("EmaCrossAlgorithm defini")
print("\nEmaCrossAlphaModel:")
print("  self.SetAlpha(EmaCrossAlphaModel(")
print("      fastPeriod=12,")
print("      slowPeriod=26,")
print("      resolution=Resolution.Daily")
print("  ))")

### 3.2 MacdAlphaModel

Le **MacdAlphaModel** utilise l'indicateur MACD (Moving Average Convergence Divergence) :

- **Signal haussier** : MACD line croise au-dessus de la Signal line
- **Signal baissier** : MACD line croise en-dessous de la Signal line

**MACD Components** :
- MACD Line = EMA(fast) - EMA(slow)
- Signal Line = EMA(MACD Line)
- Histogram = MACD Line - Signal Line

In [ ]:
# MacdAlphaModel - MACD crossover strategy
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *

class MacdAlgorithm(QCAlgorithm):
    """
    Strategie utilisant MacdAlphaModel.
    Parametres MACD classiques: 12, 26, 9.
    """
    
    def Initialize(self):
        self.SetStartDate(2022, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        # Universe
        symbols = [Symbol.Create(ticker, SecurityType.Equity, Market.USA) 
                   for ticker in ["SPY", "QQQ", "IWM", "DIA"]]
        self.SetUniverseSelection(ManualUniverseSelectionModel(symbols))
        
        # === MACD ALPHA MODEL ===
        # Parametres:
        #   fastPeriod: EMA rapide pour MACD (defaut: 12)
        #   slowPeriod: EMA lente pour MACD (defaut: 26)
        #   signalPeriod: EMA pour Signal line (defaut: 9)
        #   movingAverageType: Type de MA (defaut: Exponential)
        #   resolution: Resolution (defaut: Daily)
        self.SetAlpha(MacdAlphaModel(
            fastPeriod=12,
            slowPeriod=26,
            signalPeriod=9,
            movingAverageType=MovingAverageType.Exponential,
            resolution=Resolution.Daily
        ))
        
        # Portfolio et Execution
        self.SetPortfolioConstruction(EqualWeightingPortfolioConstructionModel())
        self.SetExecution(ImmediateExecutionModel())
        self.SetRiskManagement(NullRiskManagementModel())
        
        self.Log("MACD Algorithm initialized")
        self.Log("  MACD(12, 26, 9)")

print("MacdAlgorithm defini")
print("\nMacdAlphaModel:")
print("  self.SetAlpha(MacdAlphaModel(")
print("      fastPeriod=12,")
print("      slowPeriod=26,")
print("      signalPeriod=9,")
print("      movingAverageType=MovingAverageType.Exponential")
print("  ))")

### 3.3 RsiAlphaModel

Le **RsiAlphaModel** utilise le Relative Strength Index pour detecter les conditions de surachat et survente :

- **Signal haussier** : RSI < 30 (survente) puis remonte
- **Signal baissier** : RSI > 70 (surachat) puis redescend

**RSI Formula** :
```
RSI = 100 - (100 / (1 + RS))
RS = Average Gain / Average Loss
```

In [ ]:
# RsiAlphaModel - RSI overbought/oversold strategy
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *

class RsiAlgorithm(QCAlgorithm):
    """
    Strategie utilisant RsiAlphaModel.
    RSI 14 periodes avec seuils 30/70.
    """
    
    def Initialize(self):
        self.SetStartDate(2022, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        # Universe: secteur financier
        symbols = [Symbol.Create(ticker, SecurityType.Equity, Market.USA) 
                   for ticker in ["JPM", "BAC", "WFC", "GS", "MS"]]
        self.SetUniverseSelection(ManualUniverseSelectionModel(symbols))
        
        # === RSI ALPHA MODEL ===
        # Parametres:
        #   period: Periode du RSI (defaut: 14)
        #   resolution: Resolution (defaut: Daily)
        self.SetAlpha(RsiAlphaModel(
            period=14,
            resolution=Resolution.Daily
        ))
        
        # Portfolio et Execution
        self.SetPortfolioConstruction(EqualWeightingPortfolioConstructionModel())
        self.SetExecution(ImmediateExecutionModel())
        self.SetRiskManagement(NullRiskManagementModel())
        
        self.Log("RSI Algorithm initialized")
        self.Log("  RSI(14), Overbought: 70, Oversold: 30")

print("RsiAlgorithm defini")
print("\nRsiAlphaModel:")
print("  self.SetAlpha(RsiAlphaModel(")
print("      period=14,")
print("      resolution=Resolution.Daily")
print("  ))")
print("\nLogique interne:")
print("  - RSI < 30 (oversold) -> Insight.Up")
print("  - RSI > 70 (overbought) -> Insight.Down")

### Tableau recapitulatif des Alpha Models built-in

| Model | Signal Haussier | Signal Baissier | Parametres Cles |
|-------|----------------|-----------------|------------------|
| **EmaCrossAlphaModel** | Fast EMA > Slow EMA | Fast EMA < Slow EMA | fastPeriod, slowPeriod |
| **MacdAlphaModel** | MACD > Signal | MACD < Signal | fastPeriod, slowPeriod, signalPeriod |
| **RsiAlphaModel** | RSI < 30 (oversold) | RSI > 70 (overbought) | period |

### Insight Duration des models built-in

La duree des Insights generes est generalement basee sur la resolution :

- **Daily** : Insight valide ~1 jour de trading
- **Hour** : Insight valide quelques heures
- **Minute** : Insight valide quelques minutes

---

## Partie 4 : Custom Alpha Models (30 min)

Les Alpha Models built-in sont utiles pour debuter, mais les strategies reelles necessitent souvent des modeles personnalises. Voyons comment creer nos propres Alpha Models.

### Structure d'un Custom Alpha Model

Un Alpha Model personnalise doit heriter de `AlphaModel` et implementer :

```python
class MyAlphaModel(AlphaModel):
    def __init__(self, params...):
        # Initialisation
    
    def Update(self, algorithm, data):
        # Generer et retourner des Insights
        return insights
    
    def OnSecuritiesChanged(self, algorithm, changes):
        # Reagir aux changements d'univers
```

### 4.1 Momentum Alpha Model

In [ ]:
# Custom Alpha Model: Momentum
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *
from datetime import timedelta

class MomentumAlphaModel(AlphaModel):
    """
    Alpha Model base sur le momentum (rendement passe).
    
    Genere:
    - Insight UP si momentum > seuil positif
    - Insight DOWN si momentum < seuil negatif
    - Pas d'insight si momentum entre les seuils
    """
    
    def __init__(self, lookback=252, resolution=Resolution.Daily, 
                 up_threshold=0.10, down_threshold=-0.10):
        """
        Initialise le Momentum Alpha Model.
        
        Parameters:
        -----------
        lookback : int
            Nombre de periodes pour calculer le momentum (defaut: 252 = 1 an)
        resolution : Resolution
            Resolution des donnees
        up_threshold : float
            Seuil pour signal haussier (defaut: +10%)
        down_threshold : float
            Seuil pour signal baissier (defaut: -10%)
        """
        self.lookback = lookback
        self.resolution = resolution
        self.up_threshold = up_threshold
        self.down_threshold = down_threshold
        
        # Dictionnaire pour stocker les donnees par symbole
        self.symbolData = {}
        
        # Duree des insights generes
        self.insight_period = timedelta(days=30)
    
    def Update(self, algorithm, data):
        """
        Appelee a chaque nouvelle barre de donnees.
        Genere des Insights bases sur le momentum.
        
        Parameters:
        -----------
        algorithm : QCAlgorithm
            Reference a l'algorithme principal
        data : Slice
            Donnees de marche actuelles
        
        Returns:
        --------
        list[Insight]
            Liste des Insights generes
        """
        insights = []
        
        for symbol, symbolData in self.symbolData.items():
            # Verifier que l'indicateur est pret
            if not symbolData.IsReady:
                continue
            
            # Verifier que les donnees existent pour ce symbole
            if not data.ContainsKey(symbol):
                continue
            
            # Recuperer le momentum (rendement en %)
            momentum = symbolData.Momentum.Current.Value
            
            # Generer les Insights selon le momentum
            if momentum > self.up_threshold:
                # Signal haussier
                insight = Insight.Price(
                    symbol,
                    self.insight_period,
                    InsightDirection.Up,
                    momentum,                    # Magnitude = momentum
                    min(momentum / 0.20, 1.0)    # Confidence proportionnelle
                )
                insights.append(insight)
                algorithm.Debug(f"UP: {symbol.Value}, Momentum={momentum:.2%}")
            
            elif momentum < self.down_threshold:
                # Signal baissier
                insight = Insight.Price(
                    symbol,
                    self.insight_period,
                    InsightDirection.Down,
                    abs(momentum),
                    min(abs(momentum) / 0.20, 1.0)
                )
                insights.append(insight)
                algorithm.Debug(f"DOWN: {symbol.Value}, Momentum={momentum:.2%}")
        
        return insights
    
    def OnSecuritiesChanged(self, algorithm, changes):
        """
        Appelee quand l'univers change (ajout/retrait de securities).
        Initialise ou nettoie les indicateurs.
        
        Parameters:
        -----------
        algorithm : QCAlgorithm
            Reference a l'algorithme
        changes : SecurityChanges
            Ajouts et retraits de securities
        """
        # Ajouter les nouvelles securities
        for security in changes.AddedSecurities:
            symbol = security.Symbol
            if symbol not in self.symbolData:
                self.symbolData[symbol] = SymbolData(algorithm, symbol, self.lookback, self.resolution)
        
        # Retirer les securities supprimees
        for security in changes.RemovedSecurities:
            symbol = security.Symbol
            if symbol in self.symbolData:
                del self.symbolData[symbol]


class SymbolData:
    """
    Classe helper pour stocker les indicateurs par symbole.
    Encapsule toutes les donnees necessaires pour un actif.
    """
    
    def __init__(self, algorithm, symbol, period, resolution):
        """
        Initialise les indicateurs pour un symbole.
        
        Parameters:
        -----------
        algorithm : QCAlgorithm
            Reference pour enregistrer les indicateurs
        symbol : Symbol
            Le symbole de l'actif
        period : int
            Periode pour le momentum
        resolution : Resolution
            Resolution des donnees
        """
        self.Symbol = symbol
        
        # Creer l'indicateur de momentum (Momentum Percent)
        # MomentumPercent = (Price - Price[n]) / Price[n]
        self.Momentum = MomentumPercent(period)
        
        # Enregistrer l'indicateur pour mise a jour automatique
        algorithm.RegisterIndicator(symbol, self.Momentum, resolution)
        
        # Warmup: charger l'historique pour initialiser l'indicateur
        history = algorithm.History(symbol, period, resolution)
        if not history.empty:
            for bar in history.itertuples():
                self.Momentum.Update(bar.Index[1], bar.close)
    
    @property
    def IsReady(self):
        """Retourne True si l'indicateur a assez de donnees."""
        return self.Momentum.IsReady


print("MomentumAlphaModel et SymbolData definis")
print("\nStructure du Custom Alpha Model:")
print("  1. __init__(): Parametres et initialisation")
print("  2. Update(): Generer des Insights")
print("  3. OnSecuritiesChanged(): Gerer les changements d'univers")

### 4.2 Utilisation du Custom Alpha Model

Voyons comment integrer notre `MomentumAlphaModel` dans un algorithme complet.

In [ ]:
# Algorithme utilisant le MomentumAlphaModel
# A copier dans l'IDE QuantConnect (avec la classe MomentumAlphaModel ci-dessus)

from AlgorithmImports import *

class MomentumFrameworkAlgorithm(QCAlgorithm):
    """
    Strategie complete utilisant le MomentumAlphaModel personnalise.
    """
    
    def Initialize(self):
        self.SetStartDate(2020, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        # Universe: ETFs sectoriels
        symbols = [Symbol.Create(ticker, SecurityType.Equity, Market.USA) 
                   for ticker in ["XLK", "XLF", "XLE", "XLV", "XLI", "XLY", "XLP", "XLU"]]
        self.SetUniverseSelection(ManualUniverseSelectionModel(symbols))
        
        # Custom Alpha Model
        self.SetAlpha(MomentumAlphaModel(
            lookback=252,           # 1 an de momentum
            resolution=Resolution.Daily,
            up_threshold=0.10,      # +10% pour signal UP
            down_threshold=-0.10    # -10% pour signal DOWN
        ))
        
        # Portfolio: allocation egale
        self.SetPortfolioConstruction(EqualWeightingPortfolioConstructionModel())
        
        # Execution immediate
        self.SetExecution(ImmediateExecutionModel())
        
        # Pas de risk management additionnel
        self.SetRiskManagement(NullRiskManagementModel())
        
        self.Log("Momentum Framework Algorithm initialized")
    
    def OnOrderEvent(self, orderEvent):
        """Log les executions d'ordres."""
        if orderEvent.Status == OrderStatus.Filled:
            self.Log(f"Order filled: {orderEvent.Symbol.Value} {orderEvent.FillQuantity} @ {orderEvent.FillPrice}")

print("MomentumFrameworkAlgorithm defini")
print("\nUsage:")
print("  self.SetAlpha(MomentumAlphaModel(lookback=252, up_threshold=0.10))")

### 4.3 Mean Reversion Alpha Model

Creons un autre Alpha Model base sur la **mean reversion** (retour a la moyenne). Cette strategie suppose que les prix reviennent vers leur moyenne apres des mouvements extremes.

In [ ]:
# Custom Alpha Model: Mean Reversion base sur Bollinger Bands
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *
from datetime import timedelta

class MeanReversionAlphaModel(AlphaModel):
    """
    Alpha Model base sur la mean reversion.
    Utilise les Bollinger Bands pour detecter les extremes.
    
    Logique:
    - Prix touche lower band -> Insight UP (survente, rebond attendu)
    - Prix touche upper band -> Insight DOWN (surachat, correction attendue)
    """
    
    def __init__(self, period=20, num_std=2.0, resolution=Resolution.Daily):
        """
        Parameters:
        -----------
        period : int
            Periode pour les Bollinger Bands
        num_std : float
            Nombre d'ecarts-types pour les bandes
        resolution : Resolution
            Resolution des donnees
        """
        self.period = period
        self.num_std = num_std
        self.resolution = resolution
        self.symbolData = {}
        self.insight_period = timedelta(days=5)  # Court terme pour mean reversion
    
    def Update(self, algorithm, data):
        insights = []
        
        for symbol, symbolData in self.symbolData.items():
            if not symbolData.IsReady:
                continue
            
            if not data.ContainsKey(symbol):
                continue
            
            price = data[symbol].Close
            bb = symbolData.BollingerBands
            
            upper = bb.UpperBand.Current.Value
            lower = bb.LowerBand.Current.Value
            middle = bb.MiddleBand.Current.Value
            
            # Calculer la position relative dans les bandes
            # %B = (Price - Lower) / (Upper - Lower)
            band_width = upper - lower
            if band_width > 0:
                percent_b = (price - lower) / band_width
            else:
                continue
            
            # Prix proche de la lower band -> signal UP (mean reversion)
            if percent_b < 0.05:  # Prix dans les 5% inferieurs
                magnitude = (middle - price) / price  # Distance a la moyenne
                insight = Insight.Price(
                    symbol,
                    self.insight_period,
                    InsightDirection.Up,
                    magnitude,
                    0.70  # Confiance moderee
                )
                insights.append(insight)
                algorithm.Debug(f"MEAN REV UP: {symbol.Value}, %B={percent_b:.2f}")
            
            # Prix proche de la upper band -> signal DOWN
            elif percent_b > 0.95:  # Prix dans les 5% superieurs
                magnitude = (price - middle) / price
                insight = Insight.Price(
                    symbol,
                    self.insight_period,
                    InsightDirection.Down,
                    magnitude,
                    0.70
                )
                insights.append(insight)
                algorithm.Debug(f"MEAN REV DOWN: {symbol.Value}, %B={percent_b:.2f}")
        
        return insights
    
    def OnSecuritiesChanged(self, algorithm, changes):
        for security in changes.AddedSecurities:
            symbol = security.Symbol
            if symbol not in self.symbolData:
                self.symbolData[symbol] = MeanReversionSymbolData(
                    algorithm, symbol, self.period, self.num_std, self.resolution
                )
        
        for security in changes.RemovedSecurities:
            symbol = security.Symbol
            if symbol in self.symbolData:
                del self.symbolData[symbol]


class MeanReversionSymbolData:
    """
    Helper class pour Mean Reversion Alpha Model.
    Stocke les Bollinger Bands pour chaque symbole.
    """
    
    def __init__(self, algorithm, symbol, period, num_std, resolution):
        self.Symbol = symbol
        
        # Creer les Bollinger Bands
        self.BollingerBands = BollingerBands(period, num_std, MovingAverageType.Simple)
        
        # Enregistrer pour mise a jour automatique
        algorithm.RegisterIndicator(symbol, self.BollingerBands, resolution)
        
        # Warmup
        history = algorithm.History(symbol, period, resolution)
        if not history.empty:
            for bar in history.itertuples():
                self.BollingerBands.Update(bar.Index[1], bar.close)
    
    @property
    def IsReady(self):
        return self.BollingerBands.IsReady


print("MeanReversionAlphaModel defini")
print("\nLogique Mean Reversion:")
print("  - Prix < Lower Band -> Acheter (survente)")
print("  - Prix > Upper Band -> Vendre (surachat)")
print("  - Insight court terme (5 jours) pour profiter du rebond")

### 4.4 Pattern SymbolData

Le pattern **SymbolData** est fondamental pour les Alpha Models personnalises. Il permet de :

1. **Encapsuler** les indicateurs et donnees par symbole
2. **Gerer proprement** le lifecycle (creation/suppression)
3. **Faciliter** le warmup des indicateurs

```
                  Alpha Model
                      |
         +------------+------------+
         |            |            |
    SymbolData   SymbolData   SymbolData
      (AAPL)       (MSFT)       (GOOGL)
         |            |            |
    Indicators   Indicators   Indicators
```

In [ ]:
# Template SymbolData avec plusieurs indicateurs
# A copier et adapter pour vos propres Alpha Models

from AlgorithmImports import *

class AdvancedSymbolData:
    """
    Template avance de SymbolData avec plusieurs indicateurs.
    Utilisable comme base pour des Alpha Models complexes.
    """
    
    def __init__(self, algorithm, symbol, resolution=Resolution.Daily):
        """
        Initialise tous les indicateurs pour un symbole.
        """
        self.Symbol = symbol
        self._algorithm = algorithm
        
        # === TREND INDICATORS ===
        self.SmaFast = SimpleMovingAverage(10)
        self.SmaSlow = SimpleMovingAverage(50)
        self.Ema = ExponentialMovingAverage(20)
        
        # === MOMENTUM INDICATORS ===
        self.Rsi = RelativeStrengthIndex(14)
        self.Macd = MovingAverageConvergenceDivergence(12, 26, 9)
        self.Momentum = MomentumPercent(20)
        
        # === VOLATILITY INDICATORS ===
        self.Atr = AverageTrueRange(14)
        self.BollingerBands = BollingerBands(20, 2)
        
        # Liste de tous les indicateurs pour enregistrement
        self._indicators = [
            self.SmaFast, self.SmaSlow, self.Ema,
            self.Rsi, self.Macd, self.Momentum,
            self.Atr, self.BollingerBands
        ]
        
        # Enregistrer tous les indicateurs
        for indicator in self._indicators:
            algorithm.RegisterIndicator(symbol, indicator, resolution)
        
        # Warmup avec historique
        self._warmup(algorithm, symbol, 100, resolution)
    
    def _warmup(self, algorithm, symbol, period, resolution):
        """Charge l'historique pour initialiser les indicateurs."""
        history = algorithm.History(symbol, period, resolution)
        if not history.empty:
            for bar in history.itertuples():
                time = bar.Index[1]
                # Update des indicateurs simples (close)
                self.SmaFast.Update(time, bar.close)
                self.SmaSlow.Update(time, bar.close)
                self.Ema.Update(time, bar.close)
                self.Rsi.Update(time, bar.close)
                self.Macd.Update(time, bar.close)
                self.Momentum.Update(time, bar.close)
                self.BollingerBands.Update(time, bar.close)
                # ATR necessite une TradeBar
                # (simplification: utilise close pour le warmup basique)
    
    @property
    def IsReady(self):
        """Tous les indicateurs doivent etre prets."""
        return all(ind.IsReady for ind in self._indicators)
    
    # === PROPRIETES DERIVEES ===
    
    @property
    def IsBullishTrend(self):
        """True si tendance haussiere (SMA fast > slow)."""
        return self.SmaFast.Current.Value > self.SmaSlow.Current.Value
    
    @property
    def IsOversold(self):
        """True si RSI indique survente."""
        return self.Rsi.Current.Value < 30
    
    @property
    def IsOverbought(self):
        """True si RSI indique surachat."""
        return self.Rsi.Current.Value > 70
    
    @property
    def MacdBullishCross(self):
        """True si MACD vient de croiser au-dessus de la signal line."""
        return self.Macd.Current.Value > self.Macd.Signal.Current.Value


print("AdvancedSymbolData template defini")
print("\nIndicateurs inclus:")
print("  - Trend: SMA(10), SMA(50), EMA(20)")
print("  - Momentum: RSI(14), MACD(12,26,9), MOM%(20)")
print("  - Volatility: ATR(14), BB(20,2)")
print("\nProprietes utiles:")
print("  - IsBullishTrend, IsOversold, IsOverbought, MacdBullishCross")

---

## Partie 5 : Composite Alpha (15 min)

### Combiner plusieurs Alpha Models

Le **CompositeAlphaModel** permet de combiner plusieurs Alpha Models en un seul. Chaque model genere ses propres Insights, et tous sont transmis au Portfolio Construction Model.

### Pourquoi combiner?

| Raison | Explication |
|--------|-------------|
| **Diversification** | Differents signaux capturent differents regimes de marche |
| **Robustesse** | Si un model echoue, les autres continuent |
| **Confirmation** | Insights multiples sur le meme actif = signal fort |
| **Flexibilite** | Tester facilement differentes combinaisons |

### Comportement du CompositeAlphaModel

```
CompositeAlphaModel
    |
    +-- EmaCrossAlphaModel  -----> [Insight AAPL UP, Insight MSFT UP]
    |
    +-- RsiAlphaModel       -----> [Insight AAPL DOWN, Insight GOOGL UP]
    |
    +-- MomentumAlphaModel  -----> [Insight MSFT UP, Insight NVDA UP]
    |
    v
    Tous les Insights combines: [AAPL UP, MSFT UP, AAPL DOWN, GOOGL UP, MSFT UP, NVDA UP]
    |
    v
    Portfolio Construction Model decide de l'allocation
```

**Note** : En cas de conflits (AAPL UP et AAPL DOWN), c'est le Portfolio Construction Model qui decide comment les reconcilier.

In [ ]:
# CompositeAlphaModel - Combiner plusieurs strategies
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *

class CompositeAlphaAlgorithm(QCAlgorithm):
    """
    Strategie combinant 3 Alpha Models:
    - EMA Cross: Trend following
    - RSI: Mean reversion
    - Momentum: Factor investing
    """
    
    def Initialize(self):
        self.SetStartDate(2022, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        # Universe large
        symbols = [Symbol.Create(ticker, SecurityType.Equity, Market.USA) 
                   for ticker in ["AAPL", "MSFT", "GOOGL", "AMZN", "META",
                                  "NVDA", "AMD", "TSLA", "JPM", "V"]]
        self.SetUniverseSelection(ManualUniverseSelectionModel(symbols))
        
        # === COMPOSITE ALPHA MODEL ===
        # Combine plusieurs Alpha Models
        self.SetAlpha(CompositeAlphaModel(
            # Model 1: EMA Cross (trend following)
            EmaCrossAlphaModel(
                fastPeriod=12,
                slowPeriod=26,
                resolution=Resolution.Daily
            ),
            # Model 2: RSI (mean reversion)
            RsiAlphaModel(
                period=14,
                resolution=Resolution.Daily
            ),
            # Model 3: MACD (momentum)
            MacdAlphaModel(
                fastPeriod=12,
                slowPeriod=26,
                signalPeriod=9,
                resolution=Resolution.Daily
            )
        ))
        
        # Portfolio Construction: allocation egale
        self.SetPortfolioConstruction(EqualWeightingPortfolioConstructionModel())
        
        # Execution
        self.SetExecution(ImmediateExecutionModel())
        
        # Risk Management
        self.SetRiskManagement(NullRiskManagementModel())
        
        self.Log("Composite Alpha Algorithm initialized")
        self.Log("  - EmaCrossAlphaModel (12, 26)")
        self.Log("  - RsiAlphaModel (14)")
        self.Log("  - MacdAlphaModel (12, 26, 9)")

print("CompositeAlphaAlgorithm defini")
print("\nUsage:")
print("  self.SetAlpha(CompositeAlphaModel(")
print("      Model1(),")
print("      Model2(),")
print("      Model3()")
print("  ))")

### Composite avec Custom Alpha Models

Vous pouvez combiner des Alpha Models built-in avec vos propres modeles personnalises.

In [ ]:
# Composite avec Custom Alpha Models
# Conceptuel - necessite les classes custom definies precedemment

from AlgorithmImports import *

print("Exemple de CompositeAlphaModel mixte:")
print("""
self.SetAlpha(CompositeAlphaModel(
    # Built-in models
    EmaCrossAlphaModel(12, 26),
    RsiAlphaModel(14),
    
    # Custom models
    MomentumAlphaModel(lookback=252, up_threshold=0.15),
    MeanReversionAlphaModel(period=20, num_std=2.0)
))
""")

print("\nAvantages de cette approche:")
print("  1. Trend following (EMA Cross) pour les tendances")
print("  2. Mean reversion (RSI, BB) pour les extremes")
print("  3. Factor (Momentum) pour la selection")
print("  4. Diversification des signaux")

---

## Partie 6 : Strategie Complete avec Framework (20 min)

### Objectif

Construire une strategie complete utilisant tous les composants du Framework :

1. **Universe** : Top 100 actions par volume
2. **Alpha** : Momentum model personnalise
3. **Portfolio** : Equal weighting
4. **Execution** : Immediate
5. **Risk** : Max 5% drawdown par security

### Architecture de la strategie

```
   Universe Selection              Alpha Model
   (Top 100 by volume)     --->    (Momentum 12M)
           |                             |
           v                             v
   ~100 securities              Insights (Up/Down)
                                         |
                                         v
                              Portfolio Construction
                                 (Equal Weight)
                                         |
                                         v
                               Target Allocations
                                         |
                                         v
                              Risk Management
                              (5% Max Drawdown)
                                         |
                                         v
                              Execution Model
                                  (Immediate)
                                         |
                                         v
                                    Orders
```

In [ ]:
# Strategie Complete avec Algorithm Framework
# A copier dans l'IDE QuantConnect

from AlgorithmImports import *
from datetime import timedelta

# === CUSTOM ALPHA MODEL ===
class QuantitativeMomentumAlpha(AlphaModel):
    """
    Alpha Model Momentum avance:
    - Momentum 12 mois (skip dernier mois pour eviter reversal)
    - Ajustement par volatilite
    - Filtre de qualite du signal
    """
    
    def __init__(self, lookback=252, skip_recent=21, min_momentum=0.05):
        self.lookback = lookback        # 12 mois
        self.skip_recent = skip_recent  # Skip 1 mois recent
        self.min_momentum = min_momentum
        self.symbolData = {}
        self.insight_period = timedelta(days=30)
    
    def Update(self, algorithm, data):
        insights = []
        
        for symbol, sd in self.symbolData.items():
            if not sd.IsReady:
                continue
            
            if not data.ContainsKey(symbol):
                continue
            
            # Momentum 12M moins 1M recent
            momentum = sd.Momentum12M
            volatility = sd.Volatility
            
            # Momentum ajuste par volatilite (Sharpe-like)
            if volatility > 0:
                risk_adjusted_momentum = momentum / volatility
            else:
                continue
            
            # Generer Insight si momentum significatif
            if momentum > self.min_momentum:
                confidence = min(risk_adjusted_momentum / 2.0, 1.0)
                confidence = max(confidence, 0.3)  # Min 30%
                
                insight = Insight.Price(
                    symbol,
                    self.insight_period,
                    InsightDirection.Up,
                    momentum,
                    confidence
                )
                insights.append(insight)
            
            elif momentum < -self.min_momentum:
                confidence = min(abs(risk_adjusted_momentum) / 2.0, 1.0)
                confidence = max(confidence, 0.3)
                
                insight = Insight.Price(
                    symbol,
                    self.insight_period,
                    InsightDirection.Down,
                    abs(momentum),
                    confidence
                )
                insights.append(insight)
        
        return insights
    
    def OnSecuritiesChanged(self, algorithm, changes):
        for security in changes.AddedSecurities:
            symbol = security.Symbol
            if symbol not in self.symbolData:
                self.symbolData[symbol] = MomentumSymbolData(
                    algorithm, symbol, self.lookback, self.skip_recent
                )
        
        for security in changes.RemovedSecurities:
            symbol = security.Symbol
            if symbol in self.symbolData:
                del self.symbolData[symbol]


class MomentumSymbolData:
    """
    Donnees pour le momentum alpha model.
    Calcule momentum 12M-1M et volatilite.
    """
    
    def __init__(self, algorithm, symbol, lookback, skip_recent):
        self.Symbol = symbol
        self.lookback = lookback
        self.skip_recent = skip_recent
        self._algorithm = algorithm
        
        # Indicateurs
        self.Std = StandardDeviation(lookback)
        algorithm.RegisterIndicator(symbol, self.Std, Resolution.Daily)
        
        # Rolling window pour calculer le momentum
        self.PriceWindow = RollingWindow[float](lookback + skip_recent)
        
        # Warmup
        history = algorithm.History(symbol, lookback + skip_recent, Resolution.Daily)
        if not history.empty:
            for bar in history.itertuples():
                self.PriceWindow.Add(bar.close)
                self.Std.Update(bar.Index[1], bar.close)
    
    @property
    def IsReady(self):
        return self.PriceWindow.IsReady and self.Std.IsReady
    
    @property
    def Momentum12M(self):
        """Rendement 12M en sautant le mois recent."""
        if not self.IsReady:
            return 0
        
        # Prix il y a 1 mois (skip recent)
        price_1m_ago = self.PriceWindow[self.skip_recent]
        # Prix il y a 12 mois + 1 mois
        price_13m_ago = self.PriceWindow[self.lookback + self.skip_recent - 1]
        
        if price_13m_ago > 0:
            return (price_1m_ago - price_13m_ago) / price_13m_ago
        return 0
    
    @property
    def Volatility(self):
        """Volatilite annualisee."""
        if not self.Std.IsReady:
            return 0
        return self.Std.Current.Value * (252 ** 0.5)  # Annualisee


print("QuantitativeMomentumAlpha et MomentumSymbolData definis")

In [ ]:
# Algorithme complet avec tous les composants
# A copier dans l'IDE QuantConnect (avec les classes ci-dessus)

from AlgorithmImports import *

class CompleteFrameworkStrategy(QCAlgorithm):
    """
    Strategie complete utilisant le Framework:
    - Universe: Top 100 par volume (Coarse Selection)
    - Alpha: Momentum quantitatif
    - Portfolio: Equal weighting
    - Execution: Immediate
    - Risk: Max drawdown per security
    """
    
    def Initialize(self):
        # Configuration
        self.SetStartDate(2018, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        # === 1. UNIVERSE SELECTION ===
        # Top 100 actions par dollar volume
        self.num_stocks = 100
        self.AddUniverse(self.CoarseSelectionFunction)
        self.UniverseSettings.Resolution = Resolution.Daily
        
        # === 2. ALPHA MODEL ===
        # Momentum 12M ajuste par volatilite
        self.SetAlpha(QuantitativeMomentumAlpha(
            lookback=252,       # 12 mois
            skip_recent=21,     # Skip 1 mois
            min_momentum=0.10   # 10% min
        ))
        
        # === 3. PORTFOLIO CONSTRUCTION ===
        # Allocation egale entre tous les Insights actifs
        self.SetPortfolioConstruction(
            EqualWeightingPortfolioConstructionModel(
                rebalance=Resolution.Daily  # Rebalancement quotidien
            )
        )
        
        # === 4. EXECUTION MODEL ===
        # Execution immediate au prix du marche
        self.SetExecution(ImmediateExecutionModel())
        
        # === 5. RISK MANAGEMENT ===
        # Maximum 5% de drawdown par position
        self.SetRiskManagement(
            MaximumDrawdownPercentPerSecurity(0.05)
        )
        
        # Tracking
        self.last_month = -1
        
        self.Log("Complete Framework Strategy initialized")
        self.Log(f"  Universe: Top {self.num_stocks} by volume")
        self.Log("  Alpha: Quantitative Momentum (12M-1M)")
        self.Log("  Portfolio: Equal Weighting")
        self.Log("  Risk: 5% Max Drawdown per Security")
    
    def CoarseSelectionFunction(self, coarse):
        """
        Selection mensuelle des top 100 actions par volume.
        """
        # Reselection mensuelle
        if self.Time.month == self.last_month:
            return Universe.Unchanged
        self.last_month = self.Time.month
        
        # Filtrer: prix > 10$, volume > 10M$, donnees fondamentales
        filtered = [x for x in coarse 
                    if x.HasFundamentalData
                    and x.Price > 10 
                    and x.DollarVolume > 10000000]
        
        # Trier par volume decroissant
        sorted_stocks = sorted(filtered, key=lambda x: x.DollarVolume, reverse=True)
        
        self.Log(f"Universe update: {len(sorted_stocks[:self.num_stocks])} stocks selected")
        
        return [x.Symbol for x in sorted_stocks[:self.num_stocks]]
    
    def OnSecuritiesChanged(self, changes):
        """Log les changements d'univers."""
        for security in changes.RemovedSecurities:
            self.Log(f"Removed from universe: {security.Symbol.Value}")
    
    def OnOrderEvent(self, orderEvent):
        """Log les executions."""
        if orderEvent.Status == OrderStatus.Filled:
            direction = "BUY" if orderEvent.FillQuantity > 0 else "SELL"
            self.Log(f"{direction}: {orderEvent.Symbol.Value} {abs(orderEvent.FillQuantity)} @ {orderEvent.FillPrice:.2f}")
    
    def OnEndOfAlgorithm(self):
        """Resume final."""
        self.Log("="*60)
        self.Log("COMPLETE FRAMEWORK STRATEGY - SUMMARY")
        self.Log("="*60)
        self.Log(f"Final Portfolio Value: ${self.Portfolio.TotalPortfolioValue:,.2f}")
        total_return = (self.Portfolio.TotalPortfolioValue - 100000) / 100000
        self.Log(f"Total Return: {total_return:.1%}")
        self.Log("="*60)

print("CompleteFrameworkStrategy defini")
print("\nCette strategie combine:")
print("  1. Universe dynamique (Top 100 par volume)")
print("  2. Alpha Model momentum quantitatif")
print("  3. Portfolio equal-weight")
print("  4. Execution immediate")
print("  5. Risk management (5% max drawdown)")

### Resume des composants utilises

| Composant | Model | Description |
|-----------|-------|-------------|
| **Universe** | Coarse Selection | Top 100 par DollarVolume, mise a jour mensuelle |
| **Alpha** | QuantitativeMomentumAlpha | Momentum 12M-1M ajuste par volatilite |
| **Portfolio** | EqualWeightingPortfolioConstructionModel | Allocation egale |
| **Execution** | ImmediateExecutionModel | Execution immediate |
| **Risk** | MaximumDrawdownPercentPerSecurity | 5% max drawdown |

---

## Conclusion et Prochaines Etapes

### Recapitulatif

Dans ce notebook, nous avons couvert :

1. **Algorithm Framework** :
   - Architecture modulaire avec 5 composants
   - Avantages: reutilisabilite, testabilite, separation des responsabilites

2. **Insights** :
   - Format de communication entre Alpha et Portfolio Construction
   - Properties: Symbol, Direction, Period, Magnitude, Confidence

3. **Alpha Models Built-In** :
   - EmaCrossAlphaModel, MacdAlphaModel, RsiAlphaModel
   - Configuration et parametres

4. **Custom Alpha Models** :
   - Structure: `__init__`, `Update`, `OnSecuritiesChanged`
   - Pattern SymbolData pour gestion des indicateurs
   - Exemples: Momentum, Mean Reversion

5. **CompositeAlphaModel** :
   - Combiner plusieurs Alpha Models
   - Diversification des signaux

6. **Strategie Complete** :
   - Integration de tous les composants
   - Universe dynamique + Alpha personnalise + Risk Management

### Points Cles a Retenir

| Concept | Point Cle |
|---------|----------|
| **Framework** | 5 composants modulaires interconnectes |
| **Insights** | Direction + Period + Magnitude + Confidence |
| **Custom Alpha** | Heriter de AlphaModel, implementer Update() |
| **SymbolData** | Encapsuler indicateurs par symbole |
| **Composite** | Combiner pour diversifier les signaux |

### Tableau des Models disponibles

| Type | Models Built-In |
|------|----------------|
| **Universe** | ManualUniverseSelectionModel, CoarseFundamentalUniverseSelectionModel |
| **Alpha** | EmaCrossAlphaModel, MacdAlphaModel, RsiAlphaModel, CompositeAlphaModel |
| **Portfolio** | EqualWeightingPortfolioConstructionModel, InsightWeightingPortfolioConstructionModel |
| **Execution** | ImmediateExecutionModel, VolumeWeightedAveragePriceExecutionModel |
| **Risk** | NullRiskManagementModel, MaximumDrawdownPercentPerSecurity |

### Limitations

- **Complexite** : Le Framework ajoute de la complexite pour les strategies simples
- **Latence** : Le passage entre composants peut ajouter de la latence
- **Conflits** : Les Insights contradictoires doivent etre geres par le Portfolio Construction

### Prochaines Etapes

Pour aller plus loin avec l'Algorithm Framework :

1. **Portfolio Construction Models avances** : Mean-Variance Optimization, Risk Parity
2. **Execution Models sophistiques** : TWAP, VWAP, Implementation Shortfall
3. **Risk Management** : VaR-based, Correlation constraints
4. **Machine Learning Alpha** : Integrer des modeles ML dans les Alpha Models

### Ressources Complementaires

- [Algorithm Framework Documentation](https://www.quantconnect.com/docs/v2/writing-algorithms/algorithm-framework)
- [Alpha Models Documentation](https://www.quantconnect.com/docs/v2/writing-algorithms/algorithm-framework/alpha)
- [Insight Documentation](https://www.quantconnect.com/docs/v2/writing-algorithms/algorithm-framework/alpha/insight)
- [Custom Alpha Model Tutorial](https://www.quantconnect.com/tutorials/strategy-library/momentum-effect)

---

**Notebook complete. Vous maitrisez maintenant l'Algorithm Framework et les Alpha Models de QuantConnect.**